In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import torch
from torch import nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, fixed

# PyTorch Tutorial

In [ ]:
# Create tensor from list below and save it as l
data = [[1, 2], [3, 4]]

In [ ]:
# Create tensor with shape (1, 7) without care its value using unsigned integer 8 bits

In [ ]:
# Create tensor with shape (2, 3, 3, 4) with all zeros and store it to varible t using float 32 bit

In [ ]:
# Change elem [1, 2, 0, 1] of t to 20

In [ ]:
# Change a first row of l to be 7

In [ ]:
# Add every elem in l by 1

In [ ]:
# Create a random 2 (3, ) vector, then do elememt wise multiplication

In [ ]:
# Create a random (3, ) vector , then calculate the dot product

In [ ]:
# Create a identity (3, 3) matrix and (3, ) vector, then calculate the matrix-vector product

In [ ]:
# Create a identity (3, 3) matrix and (3, 3) matrix, then calculate the matrix-matrix product

In [ ]:
# Create a one hot encoding for a tensor [0, 1, 2, 5, 4, 0]
# https://pytorch.org/docs/stable/generated/torch.nn.functional.one_hot.html

In [ ]:
# idk, read the docs https://pytorch.org/docs/stable/tensors.html

# Working on Regression problem using Simple Linear regression

## Def

In [ ]:
def find_best_param_for_linear_regression(x, y):
    # solve it analytically
    # https://en.wikipedia.org/wiki/Simple_linear_regression

    x_bar = torch.mean(x)
    y_bar = torch.mean(y)
    w = torch.sum(torch.multiply((x - x_bar), (y - y_bar))) / torch.sum((x - x_bar)**2)
    b = y_bar - w * x_bar
    return w, b

In [ ]:
def linear_regression_forward(x, w, b):
    return w*x + b

In [ ]:
def plot_linear_regression(w, b, x, y):
    plt.scatter(x, y, alpha=0.5, label="Data")
    plt.plot([-2, 30], [linear_regression_forward(-2, w, b), linear_regression_forward(30, w, b)], c="orange", label=f"Set w={w} b={b}");
    plt.xlabel("Quiz")
    plt.ylabel("Final")
    plt.legend()
    plt.show()

In [ ]:
def calculate_error(y_true, y_pred):
    return torch.mean((y_true - y_pred)**2)
    # e = 0
    # for y_t, y_p in zip(y_true, y_pred):
    #     e += (y_t - y_p)**2
    # return e

## Load data

In [ ]:
# read csv data
df = pd.read_csv("student-score-data-reduce.csv")

In [ ]:
x = torch.from_numpy(df["Quiz"].values)
y = torch.from_numpy(df["Final"].values)
x, x_val, y, y_val = train_test_split(x, y, test_size=0.4) # 60% train
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5) # 20% validate, 20% test

In [ ]:
print(x.shape, y.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

In [ ]:
plt.scatter(x, y, label="training data");
plt.scatter(x_val, y_val, label="validation data");
plt.scatter(x_test, y_test, label="validation data");
plt.xlabel("Quiz")
plt.ylabel("Final")
plt.legend()

In [ ]:
# Try model tuning by hand
w = widgets.FloatSlider(value=0, min=-10.,max=10.0, step=0.1,)
b = widgets.FloatSlider(value=0, min=-10.,max=10.0, step=0.1,)
interact(plot_linear_regression, w=w, b=b, x=fixed(x), y=fixed(y));

## Optimize the model analytically

In [ ]:
best_w, best_b = find_best_param_for_linear_regression(x, y)
print(best_w, best_b)

In [ ]:
calculate_error(y, linear_regression_forward(x, best_w, best_b))

In [ ]:
plot_linear_regression(best_w, best_b, x, y)

## Use PyTorch to Optimize the model numerically

In [ ]:
class ScoreDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        assert len(x) == len(y)
        self.n = len(x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    def __len__(self):
        return self.n

In [ ]:
class SimpleLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.w = Parameter(torch.empty(1, dtype=torch.float64))
        self.b = Parameter(torch.empty(1, dtype=torch.float64))

    def forward(self, x):
        return self.w * x + self.b

In [ ]:
def train_model_one_epoch(model, train_dataloader, optimizer, loss_fn):
    model.train()
    cummulative_loss = 0
    for data in train_dataloader:
        # Every data instance is an input + label pair
        inputs, labels = data
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        cummulative_loss += loss.item()

    cummulative_loss /= len(train_dataloader)
    return cummulative_loss

In [ ]:
@torch.no_grad()
def evaluate_model(model, dataloader):
    model.eval()
    cummulative_loss = 0
    for data in dataloader:
        # Every data instance is an input + label pair
        inputs, labels = data

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss
        loss = loss_fn(outputs, labels)
        cummulative_loss += loss.item()

    cummulative_loss /= len(dataloader)
    return cummulative_loss

### Train

In [ ]:
# Create dataset
train_dataset = ScoreDataset(x, y)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = ScoreDataset(x_val, y_val)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = ScoreDataset(x_test, y_test)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Create model
model = SimpleLinear()
model.train();

In [ ]:
# Create error function and optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0002)
n_epochs = 20

In [ ]:
train_loss = []
val_loss = []
for ep in range(n_epochs):
    train_loss_ep = train_model_one_epoch(model, train_dataloader, optimizer, loss_fn)
    train_loss.append(train_loss_ep)
    val_loss_ep = evaluate_model(model, val_dataloader)
    val_loss.append(val_loss_ep)
    print(f"epoch {ep}: train loss: {train_loss_ep}, val loss: {val_loss_ep}")

In [ ]:
plt.plot(train_loss, label="training loss")
plt.plot(val_loss, label="validation loss")
plt.legend()

In [ ]:
state_dict = model.state_dict()

In [ ]:
state_dict["w"], state_dict["b"]

In [ ]:
plt.title("Compare both linear regression from two methods")
plt.scatter(x, y, alpha=0.5, label="Data")
plt.plot([0, 25], [linear_regression_forward(0, best_w, best_b), linear_regression_forward(25, best_w, best_b)], c="red", label="Find parameters analytically");
plt.plot([0, 25], [model(0).detach(), model(25).detach()], c="green", label="Find parameters numerically");
plt.xlabel("Quiz")
plt.ylabel("Final")
plt.legend()

In [ ]:
# model evaluation
model.eval()
with torch.no_grad():
    print(loss_fn(model(x_test), y_test))

# Classification on MNIST dataset

## Create model from PyTorch's nn API

In [ ]:
from pathlib import Path
import requests
import pickle
import gzip

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "https://github.com/pytorch/tutorials/raw/main/_static/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)


with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
    ((x_train, y_train), (x_val, y_val), (x_test, y_test)) = pickle.load(f, encoding="latin-1")

plt.imshow(x_train[0].reshape((28, 28)), cmap="gray")
try:
    import google.colab
except ImportError:
    plt.show()
x_train, y_train, x_val, y_val, x_test, y_test = map(
    torch.tensor, (x_train, y_train, x_val, y_val, x_test, y_test)
)
x_train = x_train.float()
x_val = x_val.float()
x_test = x_test.float()

# y_train, y_val, y_test = map(
#     lambda y: F.one_hot(y, num_classes=10), (y_train, y_val, y_test)
# )

# y_train = y_train.float()
# y_val = y_val.float()
# y_test = y_test.float()

n, c = x_train.shape
print(x_train, y_train)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)
print(y_train.min(), y_train.max())

In [ ]:
class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # self.linear1 = nn.Linear(28*28, 256)
        # self.linear2 = nn.Linear(256, 128)
        # self.linear3 = nn.Linear(128, 64)
        # self.linear4 = nn.Linear(64, 10)

        self.linear1 = nn.Linear(28*28, 16)
        self.linear2 = nn.Linear(16, 16)
        self.linear3 = nn.Linear(16, 16)
        self.linear4 = nn.Linear(16, 10)
    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = self.linear4(x)
        # x = F.softmax(x, dim=1)
        return x

In [ ]:
train_dataset = ScoreDataset(x_train, y_train)
val_dataset = ScoreDataset(x_val, y_val)
test_dataset = ScoreDataset(x_test, y_test)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=256, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

In [ ]:
model = SimpleClassifier()
model.train()
epoch_idx = 0

In [ ]:
# Create error function and optimizer
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.02)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
n_epochs = 10

In [ ]:
train_loss = []
val_loss = []
for ep in range(n_epochs):
    train_loss_ep = train_model_one_epoch(model, train_dataloader, optimizer, loss_fn)
    train_loss.append(train_loss_ep)
    val_loss_ep = evaluate_model(model, val_dataloader)
    val_loss.append(val_loss_ep)
    print(f"epoch {epoch_idx}: train loss: {train_loss_ep}, val loss: {val_loss_ep}")
    epoch_idx += 1

In [ ]:
plt.plot(train_loss, label="training loss")
plt.plot(val_loss, label="validation loss")
plt.legend()

In [ ]:
model.eval()
# disable gradient calculation
with torch.no_grad():
    y_pred = model(x_train).argmax(dim=-1)
train_acc = torch.sum(y_pred == y_train)
final_train_acc = train_acc/x_train.shape[0]
print(final_train_acc)

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(x_val).argmax(dim=-1)
val_acc = torch.sum(y_pred == y_val)
final_val_acc = val_acc/x_val.shape[0]
print(final_val_acc)

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(x_test).argmax(dim=-1)
test_acc = torch.sum(y_pred == y_test)
final_test_acc = test_acc/x_test.shape[0]
print(final_test_acc)